## AutoGluon training on SageMaker

- Built-in pre-trained model ([리스트](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html))에 classification/regression 용도의 autogluon image 만 존재합니다.
- Multimodal 용도로는 직접 Container를 만들거나 Autogluon deep learnig container image ([참고](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#autogluon-training-containers)) 활용하면 됩니다.
    - 예시 : https://auto.gluon.ai/stable/tutorials/cloud_fit_deploy/cloud-aws-sagemaker-training.html
    - 예시코드 : https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/autogluon-tabular-containers


### BYOC
- AutoGluon의 경우 BYOS 모드로 requirements.txt 등의 방법으로 설치 시 시간이 오래 걸리며 매번 설치 시에 소요되는 시간이 있으므로, Container를 사용하는 게 좋습니다.
- DLC가 이미 있다면 BYOS로 해도 상관 없지만, DLC에서 지원하지 않는 경우 (예를 들어 object detection이 0.7 stable에서는 동작안되는 문제가 있음.) container를 직접 빌드하는 방법이 있습니다.
- 여기서는 기존에 Autogluon 0.7을 DLC 만들 때 사용한 [Dockerfile](https://github.com/aws/deep-learning-containers/blob/master/autogluon/training/docker/0.7/py3/Dockerfile.cpu)을 수정하여 나만의 컨테이너를 만든 후, 기존의 코드들을 수정해서 활용해 보겠습니다.
- 참고로 GPU용 DLC는 [Dockerfile.gpu](https://github.com/aws/deep-learning-containers/blob/master/autogluon/training/docker/0.7/py3/cu117/Dockerfile.gpu)를 참고하였습니다.



In [ ]:
import sagemaker
from sagemaker import utils

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session._region_name

bucket = sagemaker_session.default_bucket()
output_path = f"s3://{bucket}/ag-od-{utils.sagemaker_timestamp()}"

In [ ]:
container = "ag-od-training" 
tag = "latest"

In [ ]:
! ./build_and_push.sh $container $tag docker/Dockerfile.ag.cpu

In [ ]:
container_gpu = "ag-od-training-gpu"
tag = "ag-gpu-0.7-src"

In [ ]:
! ./build_and_push.sh $container_gpu $tag docker/Dockerfile.ag.gpu

- 위의 과정을 이후 ECR repo에 docker image가 정상적으로 올라갔는지 확인합니다.
- push 가 실패하는 경우 ECR push 에 적절한 권한이 없을 경우이므로 ECR repository의 permission에서 SageMaker role에 대한 권한을 줄 수 있도록 합니다.

In [ ]:
# pip install -U sagemaker

In [ ]:
import sys
sys.path.append("./ag_helper")

from ag_helper.ag_model import (
    AutoGluonSagemakerEstimator,
    AutoGluonNonRepackInferenceModel,
    AutoGluonSagemakerInferenceModel,
    AutoGluonRealtimePredictor,
    AutoGluonBatchPredictor,
)

In [ ]:
# ECR push한 container image uri 값으로 변경합니다.
# container_uri = "784880277394.dkr.ecr.ap-northeast-2.amazonaws.com/ag-od-training:latest"
container_uri = "784880277394.dkr.ecr.ap-northeast-2.amazonaws.com/ag-od-training-gpu:ag-gpu-0.7-src"

In [ ]:
ag = AutoGluonSagemakerEstimator(
    custom_image_uri=container_uri,
    role=role,
    entry_point="train.py",
    source_dir="src",
    region=region,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    # instance_type="ml.m5.4xlarge",
    # instance_type="local",
    framework_version="0.7",
    py_version="py39",
    base_job_name="autogluon-od-train",
    volume_size=120,
    disable_profiler=True,
    debugger_hook_config=False,
)

In [ ]:
train_input = f"s3://{bucket}/object_detection_data/pothole/"

In [ ]:
print(output_path)
print(train_input)

In [ ]:
job_name = utils.unique_name_from_base("autogluon-od-finetune")
ag.fit(
    {
        "train": train_input,
    },
    job_name=job_name,
)